#  Projeto Deep Learning 

## Importações

In [30]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from typing import TypeVar
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.datasets import cifar10
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


## Carregamento do CSV de Metadados e verificação de imagens

In [31]:
metadata_path = "metadata.csv" 
meta_data = pd.read_csv(metadata_path)

base_dir = "/Users/joaosantos/Documents/Mestrado Joao/2 semestre/Deep Learning/rare_species 1" 

In [32]:
def check_image_exists(image_file):
    for root, dirs, files in os.walk(base_dir):
        if image_file in files:
            return True
    return False

Verificar se as imagens mencionada no Csv estão presentes nas pastas

In [ ]:
missing_images = []
for index, row in meta_data.iterrows():
    image_path = row['file_path']  
    image_name = os.path.basename(image_path)  
    if not check_image_exists(image_name):
        missing_images.append(image_path)

print(f"Número de imagens em falta: {len(missing_images)}")

Aqui é que conta

In [ ]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf

def load_local_images(base_dir, image_size=(32, 32)):
    """
    Carrega imagens a partir de um diretório e as prepara para o treinamento.

    Parâmetros:
    - base_dir (str): Caminho do diretório onde as imagens estão armazenadas.
    - image_size (tuple): Tamanho para redimensionamento das imagens.

    Retorna:
    - X_data (np.array): Imagens normalizadas.
    - y_data (np.array): Rótulos One-Hot encoded.
    """
    X_data = []
    y_data = []
    
    # Obter as classes (nomes das pastas) e garantir que são pastas de imagens
    labels = os.listdir(base_dir)
    labels = [label for label in labels if os.path.isdir(os.path.join(base_dir, label)) and not label.startswith('.')]  # Ignora arquivos como '.DS_Store'
    
    # Criar um mapeamento de classe (nome da pasta) para um índice inteiro
    label_map = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        class_dir = os.path.join(base_dir, label)
        if os.path.isdir(class_dir):  # Verificar se é um diretório
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                # Carregar a imagem
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Erro ao carregar a imagem: {img_path}")
                    continue  # Ignorar a imagem se não for carregada corretamente
                # Redimensionar a imagem para 32x32
                img = cv2.resize(img, image_size)
                X_data.append(img)
                y_data.append(label_map[label])  # O rótulo é o índice da pasta (classe)
    
    # Converter listas para numpy arrays e normalizar
    X_data = np.array(X_data, dtype="float32") / 255.0
    y_data = np.array(y_data)
    
    # One-hot encoding para os rótulos
    num_classes = len(labels)  # Aqui, pegamos dinamicamente o número de classes
    y_data = to_categorical(y_data, num_classes=num_classes)  # One-hot encoding com o número correto de classes

    return X_data, y_data

# Caminho para o diretório com as imagens
base_dir = "/Users/joaosantos/Documents/Mestrado Joao/2 semestre/Deep Learning/rare_species 1"

# Carregar as imagens locais
X_train, y_train = load_local_images(base_dir)

# Convertendo os rótulos para tf.float32 (caso necessário)
y_train = tf.cast(y_train, tf.float32)

# Verificar o número de classes
num_classes = y_train.shape[1]  # Será 203 ou o número de pastas que você tem

# Visualizar as dimensões dos dados
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, Número de classes: {num_classes}")


In [ ]:
from sklearn.model_selection import train_test_split

# Carregar as imagens
X_data, y_data = load_local_images(base_dir)

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Verifique as formas dos conjuntos de dados
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Cast labels
y_train = tf.cast(y_train, tf.float32)
y_test = tf.cast(y_test, tf.float32)

# Dataset TensorFlow
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

In [ ]:
# Ajuste no modelo para refletir o número de classes correto
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Modelo convolucional otimizado
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Agora ajustado dinamicamente para o número correto de classes
])

# Compilar o modelo
optimizer = Adam(learning_rate=0.0005)
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Treinamento
history = model.fit(
    train_dataset,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2
)

Deu 0.18 de accuracy

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Carregar o modelo salvo
modelo_final = load_model("melhor_modelo.keras")

# Previsões
y_pred_probs = modelo_final.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Matriz de confusão
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap="Blues")
plt.title("Matriz de Confusão")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()